<a href="https://colab.research.google.com/github/grupo274/desafio-04/blob/yadira/Yadira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 Agente VR - VERSÃO CORRIGIDA PARA COLAB

# === CÉLULA 1: INSTALAÇÃO E CONFIGURAÇÃO ===
!pip install pandas openpyxl google-generativeai agno -q

import pandas as pd
import numpy as np
from google.colab import files, userdata
import io
import os

# Imports do Agno
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools import tool

# Configuração da API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
model = Gemini(id="gemini-1.5-flash")

In [ ]:
# === CÉLULA 2: FUNÇÃO DE UPLOAD FINAL ===
import unicodedata
import pandas as pd
from google.colab import files
import io

def normalizar_nome_arquivo(nome):
    """Remove acentos e caracteres especiais de uma string."""
    nfkd_form = unicodedata.normalize('NFKD', nome)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

def fazer_upload():
    """Função separada para upload - execute em célula própria"""
    global dados_carregados

    print("📤 Selecionem todos os arquivos Excel necessários:")
    print("• ATIVOS")
    print("• ADMISSÃO (admitidos do mês anterior)")
    print("• APRENDIZ (para exclusão)")
    print("• ESTÁGIO (para exclusão)")
    print("• DIRETORES (para exclusão)") # <--- ADICIONADO AVISO
    print("• EXTERIOR (para exclusão)")
    print("• FÉRIAS (para exclusão)")
    print("• AFASTAMENTOS (para exclusão)")
    print("• DESLIGADOS (para regras de pagamento)")
    print("• Base sindicato x valor")
    print("• Base dias úteis")
    print("\n" + "="*50)

    uploaded = files.upload()
    dados_carregados = {}

    for filename, content in uploaded.items():
        try:
            filename_lower = filename.lower()
            filename_normalizado = normalizar_nome_arquivo(filename_lower)
            print(f"Processando: {filename} (Normalizado para: {filename_normalizado})")

            if 'ativo' in filename_normalizado:
                dados_carregados['ativos'] = pd.read_excel(io.BytesIO(content), sheet_name='ATIVOS', engine='openpyxl')
            elif 'admissao' in filename_normalizado:
                dados_carregados['admissoes'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif 'aprendiz' in filename_normalizado:
                dados_carregados['aprendiz'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif 'estagio' in filename_normalizado:
                dados_carregados['estagio'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')

            # --- NOVA REGRA ADICIONADA AQUI ---
            elif 'diretor' in filename_normalizado:
                dados_carregados['diretores'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            # ------------------------------------

            elif 'exterior' in filename_normalizado:
                dados_carregados['exterior'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif 'feria' in filename_normalizado:
                dados_carregados['ferias'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif 'afastamento' in filename_normalizado:
                dados_carregados['afastamentos'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif 'desligado' in filename_normalizado:
                dados_carregados['desligados'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif 'sindicato' in filename_normalizado or ('valor' in filename_normalizado and 'base' in filename_normalizado):
                dados_carregados['sindicato_valor'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            elif 'dia' in filename_normalizado and ('ute' in filename_normalizado or 'util' in filename_normalizado):
                dados_carregados['dias_uteis'] = pd.read_excel(io.BytesIO(content), engine='openpyxl')
            else:
                print(f"⚠️ Arquivo não reconhecido: {filename}")

        except Exception as e:
            print(f"❌ Erro ao processar {filename}: {e}")

    print(f"\n✅ {len(dados_carregados)} arquivos carregados com sucesso!")

    for nome, df in dados_carregados.items():
        print(f"• {nome}: {len(df)} linhas, {len(df.columns)} colunas")

    return dados_carregados

In [ ]:
# Execute esta célula para carregar os arquivos
dados_carregados = fazer_upload()

In [ ]:
# === CÉLULA 3: FERRAMENTAS DO AGENTE (VERSÃO FINAL) ===

@tool
def verificar_dados_carregados() -> dict:
    # (Esta função permanece inalterada)
    """Verifica se os dados foram carregados e mostra informações básicas"""
    if not dados_carregados: return {"erro": "Nenhum arquivo carregado ainda.", "instrucao": "Execute fazer_upload() em uma nova célula para carregar os arquivos."}
    info = {nome: {"total_linhas": len(df), "total_colunas": len(df.columns)} for nome, df in dados_carregados.items()}
    return {"arquivos_carregados": len(dados_carregados), "detalhes": info, "status": "✅ Dados prontos para processamento"}

@tool
def executar_calculo_vr() -> dict:
    """Executa o cálculo completo do Vale Refeição para maio/2025"""
    global resultado_vr

    if 'ativos' not in dados_carregados: return {"erro": "Base de funcionários ativos não encontrada."}

    try:
        print("🔄 Iniciando cálculo do VR...")

        # 1. PREPARAR BASE PRINCIPAL (ATIVOS + ADMISSÕES)
        ativos = dados_carregados['ativos'].copy()
        if 'admissoes' in dados_carregados:
            admissoes_df = dados_carregados['admissoes'].copy()
            col_matricula_ativos = next((col for col in ativos.columns if 'matricula' in col.lower() or 'matrícula' in col.lower()), None)
            col_matricula_admissoes = next((col for col in admissoes_df.columns if 'matricula' in col.lower() or 'matrícula' in col.lower()), None)
            if col_matricula_ativos and col_matricula_admissoes:
                admissoes_df.rename(columns={col_matricula_admissoes: col_matricula_ativos}, inplace=True)
                ativos = pd.concat([ativos, admissoes_df], ignore_index=True).drop_duplicates(subset=[col_matricula_ativos], keep='first')
                print(f"• Base de ativos após integração de {len(admissoes_df)} admissões: {len(ativos)} funcionários")

        col_matricula = next((col for col in ativos.columns if 'matricula' in col.lower() or 'matrícula' in col.lower()), None)
        if not col_matricula: return {"erro": "Coluna de matrícula não encontrada na base de ativos"}

        ativos['MATRICULA'] = pd.to_numeric(ativos[col_matricula], errors='coerce')
        ativos = ativos.dropna(subset=['MATRICULA'])
        ativos['MATRICULA'] = ativos['MATRICULA'].astype(int)

        # 2. COLETAR EXCLUSÕES
        total_exclusoes = set()
        log_exclusoes = {}
        # --- REGRA ATUALIZADA AQUI ---
        tipos_exclusao = ['aprendiz', 'estagio', 'exterior', 'ferias', 'afastamentos', 'diretores']
        # -----------------------------
        for tipo in tipos_exclusao:
            if tipo in dados_carregados:
                df_exc = dados_carregados[tipo]
                col_mat_exc = next((col for col in df_exc.columns if 'matricula' in col.lower() or 'matrícula' in col.lower()), None)
                if col_mat_exc:
                    mats_exc = set(pd.to_numeric(df_exc[col_mat_exc], errors='coerce').dropna().astype(int))
                    total_exclusoes.update(mats_exc)
                    log_exclusoes[tipo] = len(mats_exc)
                    print(f"• Exclusões ({tipo}): {len(mats_exc)}")

        # 3. PROCESSAR DESLIGADOS
        desligados_ate_15, desligados_apos_15 = set(), set()
        if 'desligados' in dados_carregados:
            desl = dados_carregados['desligados'].copy()
            col_mat_desl = next((c for c in desl.columns if 'matricula' in c.lower() or 'matrícula' in c.lower()), None)
            col_data_desl = next((c for c in desl.columns if 'data' in c.lower() and ('demiss' in c.lower() or 'deslig' in c.lower())), None)
            col_status = next((c for c in desl.columns if 'comunicado' in c.lower() or 'ok' in c.lower() or 'status' in c.lower()), None)

            if col_mat_desl and col_data_desl:
                desl['MATRICULA'] = pd.to_numeric(desl[col_mat_desl], errors='coerce')
                desl['DATA_DEMISSAO'] = pd.to_datetime(desl[col_data_desl], errors='coerce')
                desl_ok = desl[desl[col_status].astype(str).str.upper().isin(['OK', 'S', 'SIM', '1'])] if col_status else desl
                maio_2025 = desl_ok[(desl_ok['DATA_DEMISSAO'].dt.month == 5) & (desl_ok['DATA_DEMISSAO'].dt.year == 2025)].dropna(subset=['DATA_DEMISSAO', 'MATRICULA'])
                if not maio_2025.empty:
                    desligados_ate_15 = set(maio_2025[maio_2025['DATA_DEMISSAO'].dt.day <= 15]['MATRICULA'].astype(int))
                    desligados_apos_15 = set(maio_2025[maio_2025['DATA_DEMISSAO'].dt.day > 15]['MATRICULA'].astype(int))
                print(f"• Desligados até dia 15 (excluídos): {len(desligados_ate_15)}")
                print(f"• Desligados após dia 15 (proporcionais): {len(desligados_apos_15)}")

        # 4. APLICAR EXCLUSÕES TOTAIS
        total_exclusoes.update(desligados_ate_15)
        funcionarios_elegiveis = ativos[~ativos['MATRICULA'].isin(total_exclusoes)].copy()
        print(f"• Funcionários elegíveis após exclusões: {len(funcionarios_elegiveis)}")

        # 5. MAPEAR SINDICATOS, VALORES E DIAS
        col_sindicato = next((c for c in funcionarios_elegiveis.columns if 'sindicato' in c.lower()), None)
        funcionarios_elegiveis['SINDICATO'] = funcionarios_elegiveis[col_sindicato] if col_sindicato else 'GERAL'

        funcionarios_elegiveis['DIAS_UTEIS'] = 22
        if 'dias_uteis' in dados_carregados:
            du = dados_carregados['dias_uteis']
            dias_sindicato = dict(zip(du.iloc[:, 0], pd.to_numeric(du.iloc[:, 1], errors='coerce')))
            funcionarios_elegiveis['DIAS_UTEIS'] = funcionarios_elegiveis['SINDICATO'].map(dias_sindicato).fillna(22)

        funcionarios_elegiveis['VALOR_DIARIO'] = 30.0
        if 'sindicato_valor' in dados_carregados:
            sv = dados_carregados['sindicato_valor']
            valores_sindicato = dict(zip(sv.iloc[:, 0], pd.to_numeric(sv.iloc[:, 1], errors='coerce')))
            funcionarios_elegiveis['VALOR_DIARIO'] = funcionarios_elegiveis['SINDICATO'].map(valores_sindicato).fillna(30.0)

        # 6. AJUSTAR DESLIGADOS PROPORCIONAIS
        if desligados_apos_15:
            mask_proporcional = funcionarios_elegiveis['MATRICULA'].isin(desligados_apos_15)
            funcionarios_elegiveis.loc[mask_proporcional, 'DIAS_UTEIS'] = (funcionarios_elegiveis.loc[mask_proporcional, 'DIAS_UTEIS'] / 2).round().astype(int)

        # 7. CÁLCULOS FINAIS
        funcionarios_elegiveis['VR_TOTAL'] = funcionarios_elegiveis['DIAS_UTEIS'] * funcionarios_elegiveis['VALOR_DIARIO']
        funcionarios_elegiveis['VALOR_EMPRESA'] = (funcionarios_elegiveis['VR_TOTAL'] * 0.8).round(2)
        funcionarios_elegiveis['VALOR_FUNCIONARIO'] = (funcionarios_elegiveis['VR_TOTAL'] * 0.2).round(2)

        # 8. PREPARAR RESULTADO FINAL
        col_empresa = next((c for c in funcionarios_elegiveis.columns if 'empresa' in c.lower()), None)
        col_cargo = next((c for c in funcionarios_elegiveis.columns if 'cargo' in c.lower() or 'titulo' in c.lower()), None)
        colunas_resultado = ['MATRICULA', col_empresa, col_cargo, 'SINDICATO', 'DIAS_UTEIS', 'VALOR_EMPRESA', 'VALOR_FUNCIONARIO']
        resultado_vr = funcionarios_elegiveis[[c for c in colunas_resultado if c]].copy()

        rename_map = {'DIAS_UTEIS': 'DIAS A SEREM COMPRADOS', 'VALOR_EMPRESA': 'VALOR A SER COMPRADO PELA EMPRESA', 'VALOR_FUNCIONARIO': 'VALOR A SER DESCONTADO DO FUNCIONÁRIO'}
        resultado_vr.rename(columns=rename_map, inplace=True)

        total_empresa = resultado_vr['VALOR A SER COMPRADO PELA EMPRESA'].sum()
        total_geral = resultado_vr['VALOR A SER COMPRADO PELA EMPRESA'].sum() + resultado_vr['VALOR A SER DESCONTADO DO FUNCIONÁRIO'].sum()

        print("✅ Cálculo concluído com sucesso!")
        return { "sucesso": True, "funcionarios_elegiveis": len(resultado_vr), "custo_total": f"R$ {total_geral:,.2f}" }
    except Exception as e:
        import traceback
        return { "erro": f"Erro durante o cálculo: {str(e)}", "detalhes": traceback.format_exc() }



In [ ]:
agent = Agent(
    model=model,
    show_tool_calls=True,
    markdown=True,
    instructions="""
    Você é um especialista em cálculo de Vale Refeição.

    PROCESSO PARA CALCULAR VR MAIO/2025:

    1. VERIFICAR se os dados foram carregados: verificar_dados_carregados()
    2. Se não foram carregados, INSTRUIR o usuário a executar fazer_upload() em nova célula
    3. EXECUTAR o cálculo: executar_calculo_vr()
    4. EXPORTAR planilha: exportar_planilha()

    IMPORTANTE:
    - Se dados não estão carregados, NÃO tente calcular
    - Sempre explique cada etapa e mostre os resultados
    - Se houver erro, explique claramente o que aconteceu
    - Mostre estatísticas importantes (totais, médias, etc.)
    """
)

# Registrar ferramentas
agent.add_tool(verificar_dados_carregados)
agent.add_tool(executar_calculo_vr)
agent.add_tool(exportar_planilha)

print("🤖 Agente configurado!")
print("\n" + "="*60)
print("📋 COMO USAR:")
print("2. Depois execute: agent.print_response('Calcule o VR de maio/2025')")
print("="*60)

In [ ]:
agent.print_response('Calcule o VR de maio/2025')


In [ ]:
@tool
def exportar_planilha() -> dict:
    """Exporta o resultado final do cálculo do VR para um arquivo Excel."""
    global resultado_vr

    if resultado_vr is None:
        return {"erro": "Nenhum resultado de cálculo para exportar.", "instrucao": "Execute executar_calculo_vr() primeiro."}

    try:
        nome_arquivo = "resultado_calculo_vr_maio_2025.xlsx"
        resultado_vr.to_excel(nome_arquivo, index=False)
        print(f"✅ Resultado exportado para '{nome_arquivo}'")
        return {"sucesso": True, "arquivo": nome_arquivo, "instrucao": f"O arquivo '{nome_arquivo}' foi gerado. Você pode baixá-lo usando o ícone de pasta ao lado."}
    except Exception as e:
        return {"erro": f"Erro ao exportar planilha: {str(e)}"}